# Example usage

In [ ]:
%pip install coderbot_sim

In [ ]:
from coderbot_sim import RobotSim, example_maps
from IPython.display import display
import asyncio

widget = RobotSim(example_maps.gen_simple_map(), debugDraw=True, show_controls=True)
display(widget)

async def my_sim():
    print(widget.sensor())
    await widget.step(0.5, forward=True)
    print(widget.sensor())

asyncio.gather(my_sim(),)

In [ ]:
widget.reset()

# Simple _mock_ training

In [ ]:
from coderbot_sim import RobotSim, example_maps
from IPython.display import display
import asyncio
import numpy as np
import random

ACTIONS = ["forward", "left", "right"]

q_table = {}
alpha = 0.1  # learning rate
gamma = 0.9  # discount factor
epsilon = 0.1  # exploration rate

def get_state(lidar):
    # Quantize distances into bins
    distances = [np.linalg.norm(p) for p in lidar.get("hitPoints", [])[:8]]
    # Change heuristic based on hitBodies; Goal > Pushable Box > Obstacle > Wall
    bins = np.digitize(distances, [50, 100, 200])
    return tuple(bins)

def choose_action(state):
    if random.random() < epsilon or state not in q_table:
        return random.choice(ACTIONS)
    return max(q_table[state], key=q_table[state].get)

async def take_action(action):
    if action == "forward":
        await widget.step(0.1, forward=True)
    elif action == "left":
        await widget.step(0.2, left=True)
    elif action == "right":
        await widget.step(0.2, right=True)

async def train_q_learning(episodes=5):
    for ep in range(episodes):
        total_reward = 0
        for _ in range(50):
            data = widget.sensor()
            lidar = data.get("lidar", {})
            state = get_state(lidar)

            action = choose_action(state)
            await take_action(action)

            next_data = widget.sensor()
            next_state = get_state(next_data.get("lidar", {}))

            # Reward: penalize close walls
            distances = [np.linalg.norm(p) for p in lidar.get("hitPoints", [])]
            reward = -np.sum(np.array(distances) < 50) * 0.1 + 1.0

            # Update Q-table
            q_table.setdefault(state, {a: 0 for a in ACTIONS})
            q_table.setdefault(next_state, {a: 0 for a in ACTIONS})

            old_value = q_table[state][action]
            next_max = max(q_table[next_state].values())
            q_table[state][action] = old_value + alpha * (reward + gamma * next_max - old_value)

            total_reward += reward

        widget.reset()

        print(f"Episode {ep+1}: total reward = {total_reward:.2f}")

# Create and show robot
widget = RobotSim(example_maps.gen_simple_map(), debugDraw=True, show_controls=True)
display(widget)

# ✅ Run in Jupyter
await train_q_learning()
